In [1]:
# Start with importing
# imports
# deconv library
from FastLZeroSpikeInference import fast

# a vely useful library
from boltons.iterutils import remap
import itertools

# some classic things
import numpy as np
import os

# to load and save matfiles
import scipy
import mat73

# custom functions elsewhere
# import functions that I need to run
from L0_deconv import fit_lambda, run_l0events


In [2]:
# some function defination. To load files
# this function mostly copied from
# https://stackoverflow.com/questions/15436318/traversing-a-dictionary-recursively
def get_dict_item(dic,target_key = 'Fc'):
    for k,v in dic.items():
        if isinstance(v,dict):
            for k1,v1 in get_dict_item(v,target_key):
                yield (k,)+k1, v1
        else:
            if k == target_key:
                yield (k,),v

# some function definition
def get_traces(dat, target_key):
    key_chain = []
    traces = []
    # The thing with this get_dict_item function is it uses yield instead 
    # of return, so it is a generator object. 
    # So I can only repeatly run this code if I intialize it as an object.
    gen_object = get_dict_item(dat,target_key)
    for p,v in gen_object:
        key_chain += [p]
        traces += [v]

    return key_chain, traces

def traverse_dict(dat,key_chain):
    # create a dictionary from a given key chain
    ref = dat
    for i,key in enumerate(key_chain[:-1]):
        if key in ref.keys():
            ref = ref[key]
        else:
            ref[key] = {key_chain[i+1]:{}}
            ref = ref[key]
    return ref


In [3]:
# loop through the files and find what is needed (But what? I think I need the fluroscent traces?)

# goto the directory with my data files
start_dir = '/mnt/lilyDrive/Brenna_data/recordingsRenames'
save_dir = '/mnt/lilyDrive/Brenna_data/recordingsRenames_deconv'

# get a filelist.
files = [f for f in os.listdir(start_dir) 
         if f.endswith('summaryROIs.mat')]



/mnt/lilyDrive/Brenna_data/recordingsRenames/TdTG7_19_005_fld1_summaryROIs.mat


In [5]:
for i,f in enumerate(files[0:3]):
    dat = {}
    fpath = os.path.join(start_dir,f)
    new_path = os.path.join(save_dir,f[:-4] + '_deconv.mat')
    print(fpath)
    print(new_path)

/mnt/lilyDrive/Brenna_data/recordingsRenames/TdTG7_02_002_fld1_summaryROIs.mat
/mnt/lilyDrive/Brenna_data/recordingsRenames_deconv/TdTG7_02_002_fld1_summaryROIs_deconv.mat
/mnt/lilyDrive/Brenna_data/recordingsRenames/TdTG7_02_002_fld2_summaryROIs.mat
/mnt/lilyDrive/Brenna_data/recordingsRenames_deconv/TdTG7_02_002_fld2_summaryROIs_deconv.mat
/mnt/lilyDrive/Brenna_data/recordingsRenames/TdTG7_02_003_fld1_summaryROIs.mat
/mnt/lilyDrive/Brenna_data/recordingsRenames_deconv/TdTG7_02_003_fld1_summaryROIs_deconv.mat


In [6]:
#filepath = [filepath[0]]
# batch loop for later use

for i,f in enumerate(files):
    # maybe it's safer to make sure dat is clean
    dat = {}
    fpath = os.path.join(start_dir,f)
    dat = mat73.loadmat(f)
    key_chain, traces =  get_traces(dat, 'Fc')
    # temp, I just wanted to run one among those
    # key_chain = [key_chain[0]]
    # traces = [traces[0]]
    new_dat = {}
    for j, ck in enumerate(key_chain):
         # (n,t) array n#traces, t#frames
        ct = traces[j]
        if ct.shape[1] < ct.shape[0]:
            # nframes should be way larger than ntraces
            ct = ct.transpose()
        # call function in the l0_deconv.py to run deconvolution
        y = np.array(ct[:,:]) # (n,t) array n#traces, t#frames
        l0 = fit_lambda(y,event_min_size = 3) # event_min_size need more thought
        dec = run_l0events(l0,y)
        # output is a dictionary where:
        #   dec['dff'] ->   dec={
        #     'dff': The original fc trace?
        #     'est': Something with the same length as Fc.
        #     'times': index of event location,
        #     'events': estimated event size,
        #     'gamma': l0.gamma value,
        #     'lambda': l0.lambdas value,
        # }
        # Then for this traces I will try store it back to the
        # original dict.
        # ref_point = dat
        # ck = ck[:-1] # until onelayer to the last is where we insert
        # for k in ck:
        #     # getting reference to node in the dict. 
        #     # Don't think changing dict inplace is recommended, but 
        #     # trust me I know what I am doing. 
        #     ref_point = ref_point[k]
        # Try to make a new dict with the same keys
        ref_point = traverse_dict(new_dat,ck)
        ref_point['Fc'] = dec['dff']
        ref_point['est'] = dec['est']
        # has to make these equal length cuz otherwise scipy is mad.
        # How did I do this? Idk.
        # Man on stakeoverflow did this.
        ref_point['times'] = list(zip(*itertools.zip_longest(*dec['times'],
                                                        fillvalue = np.nan)))
        ref_point['events'] = list(zip(*itertools.zip_longest(*dec['events'],
                                                        fillvalue = np.nan)))
        # just in case I wantted to also save the file an equal length array
        ref_point['times_long'] = np.zeros(dec['dff'].shape)
        ref_point['events_long'] = np.zeros(dec['dff'].shape)
        for r in range(len(dec['times'])):
            c_idx = dec['times'][r]
            r_idx = np.ones(c_idx.shape,dtype='int')*r
            ref_point['times_long'][r_idx,c_idx] = 1
            ref_point['events_long'][r_idx,c_idx] = dec['events'][r]

        ref_point['l0_gama'] = dec['gamma']
        ref_point['l0_lambda'] = dec['lambda']
    # saving. Need to remove some None and just save roi dat
    # (how this work? Idk. Code Man Stackoverflow)
    # drop_keys = lambda path, key, value: (not value is None) & \
    #         (not key in set(['gui_behav','track_behav']))
    # clean_dat = remap(dat, visit = drop_keys)
    # generate new path and actually save
    new_path = os.path.join(save_dir,f[:-4] + '_deconv.mat')
    print("saved ", new_path)
    scipy.io.savemat(new_path, new_dat)


ERROR: MATLAB type not supported: table, (uint32)
ERROR: MATLAB type not supported: table, (uint32)
Computing df/f............................................................done!
Calculating events in progress
using bisection
0
right endpoint:32
finding lambda to minimize num events < min size
lambda = 0.12371826171875
.using bisection
0
right endpoint:8
finding lambda to minimize num events < min size
lambda = 0.10205078125
.using bisection
0
right endpoint:8
finding lambda to minimize num events < min size
lambda = 0.08868408203125
.using bisection
0
right endpoint:16
finding lambda to minimize num events < min size
lambda = 0.052734375
.using bisection
0
right endpoint:8
finding lambda to minimize num events < min size
lambda = 0.0693359375
.using bisection
0
right endpoint:64
finding lambda to minimize num events < min size
lambda = 0.07781982421875
.using bisection
0
right endpoint:16
finding lambda to minimize num events < min size
lambda = 0.07354736328125
.using bisection
0
ri

In [62]:

new_dat = {}
ref = new_dat
for i,key in enumerate(key_chain[0]):
    ref = {key:[1]}
print(new_dat)

{'dSPN': [], 'Fc': []}


In [63]:
key_chain

[('dSPN', 'Fc'), ('iSPN', 'Fc')]

In [49]:
dat['dSPN']['numTransients']['myPeaks'][1]

array([0.19428698, 0.16224074, 0.14812328, 0.1955226 , 0.23541128,
       0.17667876,        nan, 0.19721048, 0.18058747, 0.14467891,
              nan, 0.16530597, 0.14675497, 0.23444491,        nan,
       0.14639354, 0.16423579, 0.17352998, 0.18555731, 0.23050909,
              nan,        nan, 0.15722856, 0.33576616, 0.18229198,
       0.20061465, 0.14703272, 0.19715688, 0.79180612, 0.26083008,
       0.58508971])

In [55]:

drop_keys = lambda path, key, value: (not value is None) & \
            (key in set(['times']))
clean_dat = remap(dat, visit = drop_keys)
# generate new path and actually save
new_path = os.path.join(save_dir,files[i][:-4] + '_deconv.mat')
print(clean_dat)
scipy.io.savemat(new_path, clean_dat)

{'concatenatedBehavir': {'ballPitch': array([ 0.00135251,  0.00343859,  0.00371582, ..., -0.0016774 ,
        0.00140941,  0.00139814]), 'ballSensor1_y': array([0.00135251, 0.00343859, 0.00371582, ..., 0.00272394, 0.00221788,
       0.00139814]), 'ballSensor1_ysign': array([3.29684154, 3.29805172, 3.29693769, ..., 1.16736034, 2.89088702,
       3.29726536]), 'ignoreTrialActual': array([ 8., 18.]), 'ignoreTrialRelative': array([ 8., 18.]), 'lick': array([0., 0., 0., ..., 0., 0., 0.]), 'lick_count': array([0., 0., 0., ..., 0., 0., 0.]), 'reward': array([0., 0., 0., ..., 0., 0., 0.]), 'reward_count': array([0., 0., 0., ..., 0., 0., 0.]), 'timestamp': array([ 289.6225 ,  289.65475,  289.687  , ..., 2581.312  , 2581.3445 ,
       2581.37675]), 'trackLength': array(1.4), 'virmenInfo': {'ITIint': array([3., 6.]), 'WorldProb': array(1.), 'gateTO': array(1.), 'nTrials': array(50.), 'phaseMapping': [[array(1.), 'Wait zone'], [array(2.), 'Track on'], [array(3.), 'reward'], [array(4.), 'omission']

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (29,) + inhomogeneous part.

In [10]:
dat['gui_rois']['coreg']['dSPN']['times']

[[571,
  627,
  656,
  1108,
  1249,
  1283,
  2045,
  2129,
  2149,
  2367,
  2375,
  2380,
  2390,
  2401,
  2411,
  2426,
  2457,
  3317,
  3812,
  4495,
  4518,
  5073,
  5112,
  5143,
  5432,
  5688,
  6136,
  6386,
  6406,
  6428,
  6437,
  6440,
  6447,
  6453,
  6460,
  6469,
  6480,
  6486,
  6501,
  6651,
  7848,
  7993,
  8954,
  8976,
  9339,
  10464,
  10615,
  10619,
  10625,
  10636,
  10643,
  10655,
  10669,
  10682,
  10701,
  10973,
  11648,
  12251,
  12702,
  12713,
  13456,
  15127,
  15801,
  15871,
  16252,
  16961,
  16985,
  17025,
  17274,
  17658,
  18374,
  18466,
  18531,
  18932,
  18973],
 [245,
  703,
  902,
  905,
  3362,
  10375,
  10381,
  10396,
  10409,
  10445,
  15698,
  15727,
  17315,
  17318,
  17343,
  17445]]

In [44]:
from boltons.iterutils import remap
drop_keys = lambda path, key, value: not value is None
clean = remap(dat, visit = drop_keys)
print(clean['gui_rois']['coreg']['dSPN']['times'])

[[571, 627, 656, 1108, 1249, 1283, 2045, 2129, 2149, 2367, 2375, 2380, 2390, 2401, 2411, 2426, 2457, 3317, 3812, 4495, 4518, 5073, 5112, 5143, 5432, 5688, 6136, 6386, 6406, 6428, 6437, 6440, 6447, 6453, 6460, 6469, 6480, 6486, 6501, 6651, 7848, 7993, 8954, 8976, 9339, 10464, 10615, 10619, 10625, 10636, 10643, 10655, 10669, 10682, 10701, 10973, 11648, 12251, 12702, 12713, 13456, 15127, 15801, 15871, 16252, 16961, 16985, 17025, 17274, 17658, 18374, 18466, 18531, 18932, 18973], [245, 703, 902, 905, 3362, 10375, 10381, 10396, 10409, 10445, 15698, 15727, 17315, 17318, 17343, 17445]]


In [50]:
import itertools
aaa = clean['gui_rois']['coreg']['dSPN']['times']
aaa = [np.array(x) for x in aaa]
print(aaa)
padded = zip(*itertools.zip_longest(*aaa,fillvalue = np.nan))
print(list(padded))


[array([  571,   627,   656,  1108,  1249,  1283,  2045,  2129,  2149,
        2367,  2375,  2380,  2390,  2401,  2411,  2426,  2457,  3317,
        3812,  4495,  4518,  5073,  5112,  5143,  5432,  5688,  6136,
        6386,  6406,  6428,  6437,  6440,  6447,  6453,  6460,  6469,
        6480,  6486,  6501,  6651,  7848,  7993,  8954,  8976,  9339,
       10464, 10615, 10619, 10625, 10636, 10643, 10655, 10669, 10682,
       10701, 10973, 11648, 12251, 12702, 12713, 13456, 15127, 15801,
       15871, 16252, 16961, 16985, 17025, 17274, 17658, 18374, 18466,
       18531, 18932, 18973], dtype=int32), array([  245,   703,   902,   905,  3362, 10375, 10381, 10396, 10409,
       10445, 15698, 15727, 17315, 17318, 17343, 17445], dtype=int32)]
[(571, 627, 656, 1108, 1249, 1283, 2045, 2129, 2149, 2367, 2375, 2380, 2390, 2401, 2411, 2426, 2457, 3317, 3812, 4495, 4518, 5073, 5112, 5143, 5432, 5688, 6136, 6386, 6406, 6428, 6437, 6440, 6447, 6453, 6460, 6469, 6480, 6486, 6501, 6651, 7848, 7993, 8954

In [ ]:
# example to drop key, none vals etc. Might need to add it to main loop
# but lets see how things goes first.
drop_keys = lambda path, key, value: key not in set(['times','events'])
clean = remap(dec, visit = drop_keys)
print(clean)
scipy.io.savemat(new_path, clean)


{'dff': array([[-0.17256028, -0.12043933, -0.03149019, ...,  0.02892456,
         0.01678189,  0.02600238],
       [-0.13460609, -0.16087307, -0.08596393, ..., -0.08080758,
        -0.0555796 , -0.07573773]]), 'est': array([[0.0001, 0.0001, 0.0001, ..., 0.0001, 0.0001, 0.0001],
       [0.0001, 0.0001, 0.0001, ..., 0.0001, 0.0001, 0.0001]]), 'gamma': 0.9179854611676618, 'lambda': [0.026123046875, 0.14776611328125]}


In [40]:
drop_keys = lambda path, key, value: key in set(['times','events'])
clean = remap(dec, visit = drop_keys)
print(clean)
import json
json.dumps(clean)
print(dec['times'])

{'times': [], 'events': []}
[array([ 1247,  2288,  2543,  3411,  3417,  3427,  3437,  3449,  3459,
        3464,  3471,  3480,  3493,  3503,  3512,  3521,  3537,  3556,
        4216,  6174,  6203,  6227,  7257,  7279,  8218,  8222,  8232,
        8248,  8259,  8269,  8345,  8349,  8363,  8382,  8609,  8615,
        8628,  8660,  9402,  9405,  9425,  9450,  9475,  9679,  9691,
        9747, 10383, 10412, 10433, 10491, 10500, 10521, 10713, 10721,
       10743, 10759, 10778, 10891, 10926, 10940, 10955, 11291, 11297,
       11304, 11313, 11321, 11329, 11338, 11349, 11357, 11368, 11379,
       11392, 11404, 11418, 11433, 11457, 11989, 12015, 12989, 13014,
       13070, 13719, 14445, 14537, 14588, 14595, 14605, 14612, 14621,
       14634, 14648, 14669, 14696, 15686, 15890, 15905, 15915, 15935,
       16698, 16865, 16874, 16886, 16891, 16900, 16910, 16918, 16926,
       16936, 16950, 16961, 16972, 16987, 17015, 17563, 17570, 17576,
       17586, 17594, 17605, 17616, 17627, 17640, 17650, 17662

In [29]:
aaa = np.zeros(dec['dff'].shape).transpose()
aaa[dec['times'][0],0] = 1
aaa[dec['times'][1],1] = 1
print(dec['times'])
print(np.where(aaa[:,1] == 1))

[array([ 1247,  2288,  2543,  3411,  3417,  3427,  3437,  3449,  3459,
        3464,  3471,  3480,  3493,  3503,  3512,  3521,  3537,  3556,
        4216,  6174,  6203,  6227,  7257,  7279,  8218,  8222,  8232,
        8248,  8259,  8269,  8345,  8349,  8363,  8382,  8609,  8615,
        8628,  8660,  9402,  9405,  9425,  9450,  9475,  9679,  9691,
        9747, 10383, 10412, 10433, 10491, 10500, 10521, 10713, 10721,
       10743, 10759, 10778, 10891, 10926, 10940, 10955, 11291, 11297,
       11304, 11313, 11321, 11329, 11338, 11349, 11357, 11368, 11379,
       11392, 11404, 11418, 11433, 11457, 11989, 12015, 12989, 13014,
       13070, 13719, 14445, 14537, 14588, 14595, 14605, 14612, 14621,
       14634, 14648, 14669, 14696, 15686, 15890, 15905, 15915, 15935,
       16698, 16865, 16874, 16886, 16891, 16900, 16910, 16918, 16926,
       16936, 16950, 16961, 16972, 16987, 17015, 17563, 17570, 17576,
       17586, 17594, 17605, 17616, 17627, 17640, 17650, 17662, 17675,
       17687, 17708